In [ ]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback.provider.openai import OpenAI
from trulens_eval import Tru

In [ ]:
tru = Tru()
tru.reset_database()

In [ ]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback.provider.openai import OpenAI
import numpy as np
# from openai import OpenAI
import os

# oai_client = OpenAI()

provider = OpenAI(model_engine="gpt-4o")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name="Groundedness")
    # .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name="Context Relevance")
    .on_input()
    # .on(Select.RecordCalls.retrieve.rets[:])
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

In [ ]:
# from .... agents.agent import marl
# from .... agents.coordinator import Coordinator
from agents.agent import marl
from agents.coordinator import Coordinator

crd = Coordinator(
    team_name="test",
    agents=marl(collection_name="test_01_ubuntu"),
)



In [ ]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(crd,
    app_id = 'RAG v1',
    feedbacks = [f_groundedness, f_answer_relevance, f_context_relevance], selectors_nocheck=True)

In [ ]:
with tru_rag as recording:
    crd.code_gen_group_chat(
    "Create a simplified example of graph attention networks in python to help me grasp the context."
)

In [ ]:
tru.get_leaderboard()

In [ ]:
last_record = recording.records[-1]

from trulens_eval.utils.display import get_feedback_result
print(get_feedback_result(last_record, "Context Relevance"))

In [ ]:
recording.records

In [ ]:
tru.run_dashboard()